In [63]:
import os

# set dir by hardcode to run in crontab
os.chdir("/home/jacob/Project/LendingClub/")

import lendingclub
import Data_Helper as DH
import numpy as np
import pandas as pd
import json
import time
import sklearn
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None  # default='warn'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
# initialize config object
config = lendingclub.ConfigData("config_data.ini")

# initialize lendingclup api object
lc = lendingclub.LendingClub(config)

# initialize data transformer
transformer = DH.Transformer_full()

# initialize DataHelper
periodStart = ("Q1", "2014")
periodEnd = ("Q2", "2017")
DataHelper = DH.DataHelper(periodStart, periodEnd, transformer, lc)

### 1. Check mapping

The variable names coming from Lending Club API and training excel files are not matching. Therefore, we need to maintain the mapping between those two set of variables. The mapping is saved in "name_mapping.csv". The mapping is then saved to "map_TrainLC.txt" through json. 

Here, we check whether the mapping is still complete. If it is not, we need to update the csv file and then txt file.

In [65]:
# get raw data from lending club API
loans = lc.get_listed_loans()
loans = pd.DataFrame(loans["loans"])

In [4]:
# training data
df = pd.read_csv('Data/LoanStats_2016Q1.csv', sep=",", skiprows=1)
df = df[df["term"]== " 36 months"] #" 36 months"
df.shape

(96120, 150)

In [5]:
# check mapping
with open("Data/map_TrainLC2.txt", 'r') as f:
    mapping = json.load(f)

In [6]:
# check what is missing
missing_in_mapping, missing_in_training = [], []

for lc in loans.columns.values:
    if lc not in mapping:
        missing_in_mapping.append(lc)
    elif mapping[lc] not in df.columns.values:
        missing_in_training.append(lc)

The mapping is still complete. The variables not in mapping are irrelevant variables so should be ok.

In [7]:
print(missing_in_mapping)

['acceptD', 'addrZip', 'creditPullD', 'desc', 'disbursementMethod', 'empTitle', 'expD', 'expDefaultRate', 'fundedAmount', 'grade', 'housingPayment', 'ilsExpD', 'initialListStatus', 'investorCount', 'listD', 'memberId', 'mtgPayment', 'reviewStatus', 'reviewStatusD', 'secAppOpenActIl']


In [8]:
print(missing_in_training)

['serviceFeeRate']


#### save name_mapping.csv to json file

if anything is missing, open csv file, update the variables. And then call below to update the txt file.

In [9]:
#mapping = pd.read_csv("Data/name_mapping.csv", sep=",")
#lendingclub_vars = list(mapping["loans"])
#training_vars = list(mapping["train"])
#mapping = {lc:train for lc, train in zip(lendingclub_vars, training_vars)}

In [10]:
#with open("Data/map_TrainLC2.txt", 'w') as f:
#    json.dump(mapping, f)

### 2. Exclude variables

In [11]:
# get Quarter
def get_quarter(x): 
    if x.month<=3:
        return "Q1"
    elif x.month<=6:
        return "Q2"
    elif x.month<=9:
        return "Q3"
    else:
        return "Q4"

In [12]:
def return_files():
    
    # get all list of csv and excel files
    directory = !pwd
    directory = directory[0]
    data_dir = directory + "/Data"
    files = !ls {data_dir}
    files = [f for f in files if ".zip" not in f]

    # divide by type
    dict_file = ["Data/" + f for f in files if "Dict" in f][0]
    training_files = ["Data/" + f for f in files if "LoanStats" in f]
    target_file = ["Data/" + f for f in files if "listed" in f][0]
    
    return dict_file, training_files, target_file

In [13]:
def read_dict_file(dict_file):
    data_dict = pd.read_excel(dict_file).dropna()
    data_dict.set_index('LoanStatNew', inplace=True)
    data_dict.sort_index(inplace=True)
    print("Reading {}..., {}".format(dict_file, data_dict.shape))
    
    return data_dict

In [14]:
def transform_training_raw_data(training_file, term):

    # read file
    df = pd.read_csv(training_file, sep=",", skiprows=1)
    df = df[df["term"]== term] #" 36 months"

    # change issue_d to datetime type
    df["issue_d"] = [datetime.datetime.strptime(d, "%b-%Y") for d in df["issue_d"] if type(d) is str]

    issue_Qs = df["issue_d"].map(get_quarter)
    issue_Ys = df["issue_d"].map(lambda x: x.year)
    
    if np.max(issue_Ys) < 2016:
        return pd.DataFrame()
    #elif np.min(issue_Ys) >= 2020:
    #    return pd.DataFrame()
    #elif np.min(issue_Ys)==2016 and np.min(issue_Qs) in ["Q3","Q4"]:
    #    return pd.DataFrame()

    # get issuance quarter and year 
    df["issue_Q"] = issue_Qs
    df["issue_y"] = issue_Ys
    
    # remove live loans
    df = df[df["loan_status"]!="Current"]
    
    # defined bad loan status
    bad_loans = ["Charged Off", "Default", "In Grace Period", "Late (16-30 days)", "Late (31-120 days)"]

    # True/False if status is bad_loans
    df["bad_loan"] = np.where(df["loan_status"].map(lambda x: x in bad_loans), True, False)

    # change last_pymt_d to maximum date if it is NAN 
    ind = df["last_pymnt_d"].isna() 
    dates = df["issue_d"][ind] + datetime.timedelta(36*30)
    dates = dates.map(lambda x: x.strftime("%b-%Y"))
    df["last_pymnt_d"][ind] = dates

    return df

In [15]:
def get_summary_raw_training_data(training_files, target_cols):
    
    data_summary = []
    info_columns = {}
    
    for file_name in training_files:
        
        print("Reading.. ", file_name)
        df = transform_training_raw_data(file_name, term = " 36 months")
        
        if df.shape == (0,0):
            print("..Out of period")
            continue
        
        min_d = np.min(df["issue_d"])
        max_d = np.max(df["issue_d"])   
        min_f = " ".join([get_quarter(min_d),str(min_d.year)])
        max_f = " ".join([get_quarter(max_d),str(max_d.year)])
        
        # data summary
        data_summary.append({"file_name":file_name, "rows":df.shape[0], "cols":df.shape[1], "min_f":min_f, "max_f":max_f})
        
        # get column NAN Count
        if max_d.year < 2016:
            f_key = "-".join([min_d.strftime("%b%y"), max_d.strftime("%b%y")])
        else:
            f_key = file_name[15:21]
        
        info_columns[f_key] = df.isna().sum()/df.shape[0]
    
    data_summary = pd.DataFrame(data_summary)
    data_summary = data_summary[["file_name","rows","cols","min_f","max_f"]]
    info_columns = pd.DataFrame(info_columns)
    
    # extend it to see the stat
    row_mean = info_columns.apply(np.mean,axis=1)
    row_min  = info_columns.apply(np.min,axis=1)
    row_max  = info_columns.apply(np.max,axis=1)

    info_columns["mean"] = row_mean
    info_columns["min"]  = row_min
    info_columns["max"]  = row_max
    info_columns["available"] = [col in target_cols for col in info_columns.index.values]
    
    return data_summary, info_columns

In [16]:
# get file names
dict_file, training_files, target_file = return_files()
training_files = [x for x in training_files if "201" in x]

In [17]:
# this is what we will see in lendingclub API after var name change
target = DataHelper.get_listed_loandata()
target_cols = target.columns.values

# manually set this - this will be used to check the final status (y variable)
y_var = "bad_loan"
training_info_list = ["last_pymnt_d", "loan_status"]

Exclude irrelevant columns

In [18]:
# we need to exclude this from the training dataset and also the target dataset
target_exclude_list = [
    # irrelevant or duplicated
    "emp_title", "term", "grade", "initial_list_status", "disbursement_method", "zip_code", "title", 
    
    # same info with "fico_range_high"
    "fico_range_low",
    
    # info_list
    "id", "member_id", "url"
]

In [19]:
target_cols = [x for x in target_cols if x not in target_exclude_list]

Exclude variables which cannot be found in target columns.

In [20]:
# get high level summary of each dataset
# get stat of NAN data points for each variables + whether it is in the target dataset
data_summary, info_columns = get_summary_raw_training_data(training_files, target_cols)

Reading..  Data/LoanStats_2016Q1.csv
Reading..  Data/LoanStats_2016Q2.csv
Reading..  Data/LoanStats_2016Q3.csv
Reading..  Data/LoanStats_2016Q4.csv
Reading..  Data/LoanStats_2017Q1.csv
Reading..  Data/LoanStats_2017Q2.csv
Reading..  Data/LoanStats_2017Q3.csv
Reading..  Data/LoanStats_2017Q4.csv
Reading..  Data/LoanStats_2018Q1.csv
Reading..  Data/LoanStats_2018Q2.csv
Reading..  Data/LoanStats_2018Q3.csv
Reading..  Data/LoanStats_2018Q4.csv
Reading..  Data/LoanStats_2019Q1.csv
Reading..  Data/LoanStats_2019Q2.csv
Reading..  Data/LoanStats_2019Q3.csv
Reading..  Data/LoanStats_2019Q4.csv


In [21]:
# filter out the columns which is not available in the target_data
print(info_columns.shape)
filtered = info_columns[info_columns["available"]]
print(filtered.shape)

(153, 20)
(95, 20)


In [22]:
# check what variables have many missing data - mths_since, joint, sec_app series
filtered[filtered["min"] >= 0.2]

,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,mean,min,max,available
mths_since_last_delinq,0.476706,0.473724,0.459431,0.461654,0.478667,0.484092,0.489790,0.542220,0.559883,0.548617,0.544057,0.552535,0.557015,0.550530,0.545806,0.554390,0.517445,0.459431,0.559883,True
mths_since_last_record,0.817873,0.808511,0.798614,0.795592,0.802834,0.804151,0.846671,0.848217,0.850843,0.863811,0.865605,0.864674,0.866744,0.876785,0.880584,0.883268,0.842174,0.795592,0.883268,True
mths_since_last_major_derog,0.708968,0.714061,0.700127,0.701647,0.717780,0.719285,0.724132,0.750141,0.763707,0.754628,0.755028,0.760390,0.764886,0.763455,0.765581,0.769582,0.739587,0.700127,0.769582,True
annual_inc_joint,0.984103,0.988315,0.995683,0.968330,0.963647,0.951726,0.913017,0.882297,0.875000,0.887876,0.885803,0.890530,0.885873,0.883738,0.887262,0.896859,0.921254,0.875000,0.995683,True
dti_joint,0.984124,0.988315,0.995683,0.968330,0.963647,0.951726,0.913017,0.882297,0.875000,0.887876,0.885803,0.890530,0.885873,0.883738,0.887262,0.896859,0.921255,0.875000,0.995683,True
verification_status_joint,0.984103,0.988315,0.995683,0.968330,0.963647,0.951726,0.913017,0.882297,0.877098,0.891736,0.896113,0.901344,0.897691,0.893350,0.896243,0.907235,0.925496,0.877098,0.995683,True
mths_since_recent_bc_dlq,0.744361,0.743819,0.734959,0.741956,0.754437,0.757045,0.760089,0.790125,0.799093,0.794944,0.791400,0.796446,0.794996,0.788711,0.785106,0.786218,0.772732,0.734959,0.799093,True
mths_since_recent_revol_delinq,0.637568,0.635966,0.627324,0.631619,0.642726,0.646750,0.650900,0.696674,0.709978,0.703098,0.698543,0.703984,0.703636,0.695286,0.691425,0.699487,0.673435,0.627324,0.709978,True
revol_bal_joint,1.000000,1.000000,1.000000,1.000000,0.992030,0.951778,0.913017,0.882311,0.875000,0.887876,0.885803,0.890530,0.885873,0.883738,0.887262,0.896859,0.927005,0.875000,1.000000,True
sec_app_fico_range_low,1.000000,1.000000,1.000000,1.000000,0.992030,0.951778,0.913017,0.882297,0.875000,0.887876,0.885803,0.890530,0.885873,0.883738,0.887262,0.896859,0.927004,0.875000,1.000000,True


In [23]:
# filter out the columns if there are too many NANs
filtered = filtered[filtered["min"] < 0.2]
filtered.shape

(75, 20)

In [24]:
# get the col names
interim_cols = list(filtered.index.values)

In [25]:
# adding back the data that we can handle deptite of many NANs
# manually incldue following variables, since it might be useful if we process data properly
training_additional_include_list = [
    "mths_since_last_delinq", "mths_since_last_record", "mths_since_last_major_derog",
    "mths_since_recent_bc_dlq", "mths_since_recent_revol_delinq",
    "annual_inc_joint", "dti_joint", "revol_bal_joint", "verification_status_joint"
]

In [26]:
interim_cols = filtered.index.values
interim_cols = list(interim_cols) + training_additional_include_list

In [27]:
len(interim_cols)

84

In [28]:
print(interim_cols)

['loan_amnt', 'int_rate', 'installment', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num

### 3. Save variables

In [29]:
info_cols = ['id', 'loan_status', 'issue_d', "issue_Q", "issue_y", "bad_loan", "return", "log_return",
             'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 
             'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt']

In [30]:
results = {}
results["info_cols"] = info_cols
results["interim_cols"] = interim_cols

In [31]:
results["missing_cols"] = [
    # mild missing data
    #'mths_since_recent_bc', 'bc_open_to_buy', 'percent_bc_gt_75', 'bc_util', 'mo_sin_old_il_acct', 
    #'num_tl_120dpd_2m', 'emp_length', 'mths_since_recent_inq',

    # severely missing
    #'il_util',

    # month_since series
    'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record',
    'mths_since_rcnt_il', 'mths_since_recent_bc_dlq', 
    'mths_since_recent_revol_delinq',
    
    # joint series
    'annual_inc_joint', 'dti_joint', 'revol_bal_joint', 'verification_status_joint'
]

In [32]:
results["no_missing_cols"] = [x for x in results["interim_cols"] if x not in results["missing_cols"]]

In [33]:
with open("Data/variable_selected2.txt", 'w') as f:
    json.dump(results, f)